# Introduction
This is a sample code to roadtest the build of Pandas dataframes

In [71]:
import pandas as pd 
import matplotlib.pyplot as plt

In [72]:
# install pandas data reader
# conda install -c anaconda pandas-datareader
# import pandas_datareader as pdr

# Data wrangle
rangle the data to only show the correct columns

In [73]:
# get data 
chart_data = pd.read_csv('sample-data/test-data.csv', header=[0], index_col=[0], parse_dates=[0])

chart_data.head()


,bid,bid.1,bid.2,bid.3,ask,ask.1,ask.2,ask.3
DateTime,Open,High,Low,Close,Open,High,Low,Close
30/12/2022 12:00,33102.4,33113.9,33069.9,33079.4,33106,33117.5,33073.5,33083
30/12/2022 13:00,33078.9,33086.9,33043.9,33067.4,33082.5,33090.5,33047.5,33071
30/12/2022 14:00,33066.9,33107.5,32909,32993,33070.5,33109.9,32911.4,32995.4
30/12/2022 15:00,32993.5,33074.5,32930,32977,32995.9,33076.9,32932.4,32979.4


In [74]:
# Drop the all "ask" prices
chart_data = chart_data.drop(columns=['ask','ask.1', 'ask.2', 'ask.3'])

# Rename the headers
chart_data.rename(columns = {'bid':'Open', 'bid.1':'High', 'bid.2':'Low', 'bid.3':'Close'}, inplace =True)

# Drop the first row of data 
prices=chart_data.drop(chart_data.index[0])

prices

chart_data

,Open,High,Low,Close
DateTime,Open,High,Low,Close
30/12/2022 12:00,33102.4,33113.9,33069.9,33079.4
30/12/2022 13:00,33078.9,33086.9,33043.9,33067.4
30/12/2022 14:00,33066.9,33107.5,32909,32993
30/12/2022 15:00,32993.5,33074.5,32930,32977
...,...,...,...,...
01/03/2023 08:00,32695,32729.5,32659.5,32709
01/03/2023 09:00,32706.5,32742,32670.5,32692
01/03/2023 10:00,32692.5,32722,32671.5,32720
01/03/2023 11:00,32720.5,32735,32704.5,32710


# Plot a Candlestick graph
Building chats withplotly
https://plotly.com/python/candlestick-charts/

use this tutorial https://medium.datadriveninvestor.com/how-to-generate-a-graph-for-a-stock-price-with-python-yahoo-finance-and-plotly-7f3a36f5ce7e

add volume & a moving average etc
https://quantpy.com.au/python-for-finance/plotting-stock-market-data/

In [75]:
import plotly.graph_objects as go
from datetime import datetime

import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'

In [76]:
# Define a figure
chart = go.Figure()

# add a trace for a Candlestick graph
chart.add_trace(go.Candlestick())

# Map each column to the correct value from the downloaded data
chart.add_trace(go.Candlestick(
    x=prices.index,
    open = prices['Open'], 
    high=prices['High'], 
    low=prices['Low'], 
    close=prices['Close'], 
    name = 'market data'))


# Include shortcut buttons
chart.update_xaxes( rangeslider_visible=False, rangeselector=dict(
    buttons=list([
        dict(count=1, label='1h', step="hour", stepmode="backward"),
        dict(count=6, label='6h', step="hour", stepmode="backward"),
        dict(step="all") 
    ]) 
))
chart.show()

# adding indicators
- moving averages 


In [77]:
# Moving averages 

prices['MA50'] = prices['Close'].rolling(window=50, min_periods=0).mean()
prices['MA200'] = prices['Close'].rolling(window=200, min_periods=0).mean()
prices['MA200'].head(30)

prices.head()


,Open,High,Low,Close,MA50,MA200
30/12/2022 12:00,33102.4,33113.9,33069.9,33079.4,33079.40,33079.40
30/12/2022 13:00,33078.9,33086.9,33043.9,33067.4,33073.40,33073.40
30/12/2022 14:00,33066.9,33107.5,32909,32993,33046.60,33046.60
30/12/2022 15:00,32993.5,33074.5,32930,32977,33029.20,33029.20
30/12/2022 16:00,32977.5,33016.6,32907.5,33002.6,33023.88,33023.88


In [85]:
# Plot moving averages on the chart

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.10, subplot_titles=('Price Chart', 'Volume'), 
               row_width=[0.2, 0.7])

#Bottom chart displaying similar price data 
fig.add_trace(go.Candlestick(x=prices.index, open=prices["Open"], high=prices["High"],
                low=prices["Low"], close=prices["Close"], name="OHLC"), 
                row=1, col=1)
#Price chart - 50 day moving average 
fig.add_trace(go.Scatter(x=prices.index, y=prices["MA50"], marker_color='grey',name="MA50"), row=1, col=1)

# Sub chart - 200 day moving average
fig.add_trace(go.Scatter(x=prices.index, y=prices["MA200"], marker_color='lightgrey',name="MA200"), row=1, col=1)

# no volume data exists
# fig.add_trace(go.Bar(x=prices.index, y=prices['Volume'], marker_color='red', showlegend=False), row=2, col=1)
